<a href="https://colab.research.google.com/github/Oozankilic/web-scraping/blob/main/scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install gspread

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
import numpy as np
 
path = "/content/drive/MyDrive/case_study.csv"
dfs = pd.read_csv(path, header=None, nrows=1000)
arrayOfUrls = dfs.to_numpy()
productUrls = []
theDomain = "https://www.markastok.com"
for url in arrayOfUrls:
  if(url[0][-2] != '/'):
    url[0] = theDomain + (url[0][ :-1])
    productUrls.append(url[0])

In [6]:
import requests
from bs4 import BeautifulSoup
import re
import html

allFeatures = []
for URL in productUrls:
  theFeatures = []
  theFeatures.insert(0,URL)
  print(URL)
  page = requests.get(URL)
  soup = BeautifulSoup(page.content, 'html.parser')

  productCode = soup.find(class_="product-feature-content")
  if productCode is None:
    continue
  else:
    word_list = str(productCode).split()
    brokenProductCode = word_list[-2]
    a = 0
    productCode = ""
    for char in reversed(brokenProductCode):
      if char == ">":
        break
      productCode = char + productCode
    theFeatures.insert(1,productCode)

    productName = soup.find(id='product-name')
    if productName is None:
      theFeatures.insert(2,"")
    else:
      productName = html.unescape(str(productName))
      result = re.search('</span>\n(.*)\n</h1>', str(productName))
      productName = result.group(1)
      theFeatures.insert(2,productName)

    variants = soup.find(class_="new-size-variant fl col-12 ease variantList")
    if variants is None:
      theFeatures.insert(3,"")
    else:
      word_list = str(variants).split()
      passiveCount = 0
      variantCount = 0
      for word in word_list:
        if word == "passive\"":
          passiveCount = passiveCount + 1
        elif word == "class=\"col":
          variantCount = variantCount + 1
      availability = ((variantCount-passiveCount)/variantCount)*100
      availabilityPercantage = str(availability) + "%"
      theFeatures.insert(3,availabilityPercantage)


    offer = soup.find(class_="detay-indirim")
    if offer is None:
      theFeatures.insert(4,"")
    else:
      offer = offer.text.strip()
      theFeatures.insert(4,offer)

    salePrice = soup.find('span', class_="discountPrice")
    if salePrice is None:
      theFeatures.insert(5,"")
    else:
      salePrice = salePrice.text.strip()
      theFeatures.insert(5,salePrice)
    

    productPrice = soup.find('span', class_="currencyPrice discountedPrice")
    if productPrice is None:
      theFeatures.insert(6,"")
    else:
      productPrice = productPrice.text.strip()
      theFeatures.insert(6,productPrice)
    allFeatures.insert(0,theFeatures)

print(allFeatures)

https://www.markastok.com/buratti-suya-dayanikli-soft-shell-erkek-mont-55618k7282-bordo
https://www.markastok.com/buratti-slim-fit-suya-ve-ruzgara-dayanikli-kapusonlu-softshell-erkek-mont-5831900-siyah
https://www.markastok.com/buratti-slim-fit-fermuarli-dik-yaka-erkek-mont-556b79000-siyah
https://www.markastok.com/buratti-suya-dayanikli-soft-shell-erkek-mont-55618k7282-lacivert
https://www.markastok.com/buratti-suya-dayanikli-soft-shell-erkek-mont-55618k7023-saks
https://www.markastok.com/slazenger-moon-kosu-yuruyus-erkek-ayakkabi-sa10re230-lacivert
https://www.markastok.com/buratti-slim-fit-suya-ve-ruzgara-dayanikli-kapusonlu-softshell-erkek-mont-5831900-lacivert
https://www.markastok.com/buratti-suya-dayanikli-soft-shell-erkek-mont-55618k7023-haki
https://www.markastok.com/buratti-suya-ve-ruzgara-dayanikli-kapusonlu-softshell-erkek-mont-2011055-lacivert
https://www.markastok.com/buratti-pelus-astarli-kapusonlu-dar-kesim-erkek-mont-5761060-e-siyah
https://www.markastok.com/buratti-sl

In [7]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

sh = gc.create('Markastok | Ürün Raporu')

worksheet = gc.open('Markastok | Ürün Raporu').sheet1

range = 'A1:G' + str(len(productUrls)+1)
cell_list = worksheet.range(range)

cell_list[0].value = 'URL'
cell_list[1].value = 'SKU'
cell_list[2].value = 'Product Name'
cell_list[3].value = 'Availability'
cell_list[4].value = 'Offer'
cell_list[5].value = 'Sale Price'
cell_list[6].value = 'Product Price'

i = 7
for features in allFeatures:
  for feature in features:
    cell_list[i].value = feature
    i = i + 1

worksheet.update_cells(cell_list)

{'spreadsheetId': '1IhL3WcVU742pI3xtqrR6BZ8cljHBTxKivzwRdVxoIk4',
 'updatedCells': 4851,
 'updatedColumns': 7,
 'updatedRange': 'Sheet1!A1:G693',
 'updatedRows': 693}